In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
from radiometry.fluospectrum import FluoSpectrum

import compress.moments as moments

# 1D slices

We can treat each column or line of the reradiation matrix as an independent spectrum and reuse the whole theory on moments.

In [ ]:
s = FluoSpectrum(os.path.join('..', 'data', 'RIT', 'HERPIPIN.BFC'))

fluorescent_spectrum = s.get_pure_fluo_filtered()

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12, 5))

ax[0].plot(s.get_reflectance_wavelengths(), s.get_non_fluo())
ax[0].set_title('Non fluo reflectance')
ax[0].set_xlabel('$\\lambda\\:[\\mathrm{nm}]$')

ax[1].imshow(fluorescent_spectrum,
    origin='lower',
    extent=[s.get_excitation_wavelengths()[0], s.get_excitation_wavelengths()[-1],
            s.get_reflectance_wavelengths()[0], s.get_reflectance_wavelengths()[-1]])

ax[1].set_title('Reradiation matrix')
ax[1].set_xlabel('$\\lambda_i\\:[\\mathrm{nm}]$')
ax[1].set_ylabel('$\\lambda_o\\:[\\mathrm{nm}]$')

fig.tight_layout()

plt.show()

In [ ]:
print(len(s.get_reflectance_wavelengths()))
print(fluorescent_spectrum.shape)

fig, ax = plt.subplots(1, 1)
ax.plot(s.get_reflectance_wavelengths(), fluorescent_spectrum[:, 10])
fig.tight_layout()
plt.show()

In [ ]:
wl_excitation = s.get_excitation_wavelengths()
phase_excitation = moments.wavelengths_to_phase(wl_excitation)

base_absorb = moments.get_basis_signal_to_moments(phase_excitation)

moments_absorb = fluorescent_spectrum @ base_absorb
b_signal = moments_absorb @ np.linalg.inv(base_absorb)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12, 5))
ax[0].imshow(moments_absorb)
ax[1].imshow(b_signal)
fig.tight_layout()
plt.show()

In [ ]:
wl_reflectance = s.get_reflectance_wavelengths()
phase_reflectance = moments.wavelengths_to_phase(wl_reflectance)

base_refl = moments.get_basis_signal_to_moments(phase_reflectance)

moments_refl = np.zeros_like(fluorescent_spectrum)

for i in range(fluorescent_spectrum.shape[1]):
    moments_refl[:, i] = fluorescent_spectrum[:, i] @ base_refl

In [ ]:
fig, ax = plt.subplots(1, 1)
ax.imshow(moments_refl)
fig.tight_layout()
plt.show()

# 2D draft

In [ ]:
def compute_trigonometric_moments_2d(phase_x, phase_y, s, mirror_signal):
    n_moments_x = len(phase_x)
    n_moments_y = len(phase_y)

    phase_x = np.concatenate([[-np.pi], phase_x, [0 if mirror_signal else np.pi]])
    phase_y = np.concatenate([[-np.pi], phase_y, [0 if mirror_signal else np.pi]])

    signal = np.zeros((phase_x.shape[0], phase_y.shape[0]))

    signal[1:phase_x.shape[0] - 1, 1:phase_y.shape[0] - 1] = s

    for theta_i_idx in range(1, phase_x.shape[0] - 1):
        signal[theta_i_idx, 0] = signal[theta_i_idx, 1]
        signal[theta_i_idx, phase_y.shape[0] - 1] = signal[theta_i_idx, phase_y.shape[0] - 2]

    for theta_o_idx in range(1, phase_y.shape[0] - 1):
        signal[0, theta_o_idx] = signal[1, theta_o_idx]
        signal[phase_x.shape[0] - 1, theta_o_idx] = signal[phase_x.shape[0] - 2, theta_o_idx]

    signal[0, 0] = signal[1, 1]
    signal[phase_x.shape[0] - 1, phase_y.shape[0] - 1] = signal[phase_x.shape[0] - 2, phase_y.shape[0] - 2]

    trigonometric_moments = np.zeros((n_moments_x, n_moments_y), dtype=complex)

    for theta_i_idx in range(phase_x.shape[0] - 1):
        if phase_x[theta_i_idx] >= phase_x[theta_i_idx + 1]:
            continue
        
        theta_1 = phase_x[theta_i_idx]
        theta_2 = phase_x[theta_i_idx + 1]

        grad_theta = theta_2 - theta_1

        for phi_i_idx in range(phase_y.shape[0] - 1):
            if phase_y[phi_i_idx] >= phase_y[phi_i_idx + 1]:
                continue
            
            phi_1 = phase_y[phi_i_idx]
            phi_2 = phase_y[phi_i_idx + 1]

            grad_phi = phi_2 - phi_1

            P_11 = signal[theta_i_idx    , phi_i_idx    ]
            P_21 = signal[theta_i_idx + 1, phi_i_idx    ]
            P_12 = signal[theta_i_idx    , phi_i_idx + 1]
            P_22 = signal[theta_i_idx + 1, phi_i_idx + 1]

            a_1 = (P_21 - P_11) / grad_theta
            a_2 = (P_22 - P_12) / grad_theta
            
            b_1 = (theta_2 * P_11 - theta_1 * P_21) / grad_theta
            b_2 = (theta_2 * P_12 - theta_1 * P_22) / grad_theta

            m_tp = (a_2 - a_1) / grad_phi
            m_p  = (b_2 - b_1) / grad_phi
            m_t  = (phi_2 * a_1 - phi_1 * a_2) / grad_phi
            m_c  = (phi_2 * b_1 - phi_1 * b_2) / grad_phi

            # k > 0, j > 0
            for k in range(1, n_moments_x):
                exp_k_t1 = np.exp(-1j * k * theta_1)
                exp_k_t2 = np.exp(-1j * k * theta_2)

                for j in range(1, n_moments_y):
                    exp_j_p1 = np.exp(-1j * j * phi_1)
                    exp_j_p2 = np.exp(-1j * j * phi_2)
                    jk   = j * k
                    jkk  = jk * k
                    jjk  = j * jk
                    jjkk = jk * jk

                    trigonometric_moments[k, j] += (
                        (
                            -(m_tp * theta_2 * phi_2 + m_p * phi_2 + m_t * theta_2 + m_c) / jk
                            + 1j * (m_tp * phi_2 + m_t)                                   / jkk
                            + 1j * (m_tp * theta_2 + m_p)                                 / jjk
                            + m_tp                                                        / jjkk
                        ) * exp_k_t2 * exp_j_p2 # np.exp(-1j * j * phi_2 - 1j * k * theta_2)
                        +
                        (
                            (m_tp * theta_1 * phi_2 + m_p * phi_2 + m_t * theta_1 + m_c)  / jk
                            - 1j * (m_tp * phi_2 + m_t)                                   / jkk
                            - 1j * (m_tp * theta_1 + m_p)                                 / jjk
                            - m_tp                                                        / jjkk
                        ) * exp_k_t1 * exp_j_p2 # np.exp(-1j * j * phi_2 - 1j * k * theta_1)
                        +
                        (
                            (m_tp * theta_2 * phi_1 + m_p * phi_1 + m_t * theta_2 + m_c)  / jk
                            - 1j * (m_tp * phi_1 + m_t)                                   / jkk
                            - 1j * (m_tp * theta_2 + m_p)                                 / jjk
                            - m_tp                                                        / jjkk
                        ) * exp_k_t2 * exp_j_p1 # np.exp(-1j * j * phi_1 - 1j * k * theta_2)
                        +
                        (
                            -(m_tp * theta_1 * phi_1 + m_p * phi_1 + m_t * theta_1 + m_c) / jk
                            + 1j * (m_tp * phi_1 + m_t)                                   / jkk
                            + 1j * (m_p + m_tp * theta_1)                                 / jjk
                            + m_tp                                                        / jjkk
                        ) * exp_k_t1 * exp_j_p1 # np.exp(-1j * j * phi_1 - 1j * k * theta_1)
                    )

            # k = 0, j > 0
            for j in range(1, n_moments_y):
                trigonometric_moments[0, j] += grad_theta * (
                    (   
                        m_p                        / (j * j)
                        + 1j * (m_p * phi_2 + m_c) / j
                        + (theta_1 + theta_2) * (
                            m_tp                        / (2 * j * j)
                            + 1j * (m_tp * phi_2 + m_t) / (2 * j)
                        )
                    ) * np.exp(-1j * j * phi_2)
                    -
                    (
                        m_p                         / (j * j)
                        + 1j * (m_p *  phi_1 + m_c) / j
                        + (theta_1 + theta_2) * (
                            m_tp                        / (2 * j * j)
                            + 1j * (m_tp * phi_1 + m_t) / (2 * j)
                        )   
                    ) * np.exp(-1j * j * phi_1)
                )

            # k > 0, j = 0
            for k in range(1, n_moments_x):
                trigonometric_moments[k, 0] += grad_phi * (
                    (
                        m_t                          / (k * k)
                        + 1j * (m_t * theta_2 + m_c) / k
                        + (phi_1 + phi_2) * (
                            m_tp                          / (2 * k * k)
                            + 1j * (m_tp * theta_2 + m_p) / (2 * k)
                        )

                    ) * np.exp(-1j * k * theta_2)
                    -
                    (
                        m_t                          / (k * k)
                        + 1j * (m_t * theta_1 + m_c) / k
                        + (phi_1 + phi_2)  * (
                            m_tp                          / (2 * k * k)
                            + 1j * (m_tp * theta_1 + m_p) / (2 * k)
                        )
                    ) * np.exp(-1j * k * theta_1)
                )

            # k = 0, j = 0
            trigonometric_moments[0, 0] += (
                grad_theta * grad_phi * (
                    (theta_2 + theta_1) * (
                        m_t / 2 
                        + m_tp * (phi_2 + phi_1) / 4
                        )
                    + m_p * (phi_2 + phi_1) / 2
                    + m_c
                )
            )

    trigonometric_moments *= 0.25 / np.pi

    if mirror_signal:
        return 2. * trigonometric_moments
    else:
        return trigonometric_moments